In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yitinglin/Projects/news-emotion-analysis/en

In [2]:
content_df = pd.read_csv('../data/Train_Dataset.csv')
label_df = pd.read_csv('../data/Train_Dataset_Label.csv')
train_df = content_df.merge(label_df, on='id')
train_df = train_df.fillna('EMPTY')

In [3]:
test_df = pd.read_csv('../data/Test_Dataset.csv')
test_df = test_df.fillna('EMPTY')
test_title_data = pd.read_csv('./bert-base/test_title_word_vector.csv', header=None).values
test_content_data = pd.read_csv('./bert-base/test_content_word_vector.csv', header=None).values
test_tail_data = pd.read_csv('./bert-base/test_tail_word_vector.csv', header=None).values
X_test = np.concatenate((test_title_data, test_content_data, test_tail_data), axis=1)

In [4]:
y = train_df['label'].values
train_title_data = pd.read_csv('./bert-base/train_title_word_vector.csv', header=None).values
train_content_data = pd.read_csv('./bert-base/train_content_word_vector.csv', header=None).values
train_tail_data = pd.read_csv('./bert-base/train_tail_word_vector.csv', header=None).values
X = np.concatenate((train_title_data, train_content_data, train_tail_data), axis=1)

from sklearn.preprocessing import LabelEncoder
import keras

encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = keras.utils.np_utils.to_categorical(y)


Using TensorFlow backend.


In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(5, shuffle=True, random_state=2019)

model_list = []
score_list = []
for index, (train_index, valid_index) in enumerate(kf.split(X, y)):
    print(index)
    temp_model = []
    temp_score = []
    for i in range(5):
        X_train, X_valid, y_train, y_valid = X[train_index], X[valid_index], y[train_index], y[valid_index]


#         inputs = tf.keras.Input(shape=(768 * 3,))
#         reshape = tf.keras.layers.Reshape((128, 18))(inputs)
#         h1 = tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=(128, 18))(reshape)
#         h2 = tf.keras.layers.Conv1D(64, 3, activation='relu')(h1)
#         p1 = tf.keras.layers.MaxPooling1D(pool_size=2)(h2)
#         h3 = tf.keras.layers.Conv1D(128, 3, activation='relu')(p1)
#         p2 = tf.keras.layers.MaxPooling1D(pool_size=2)(h3)

#         bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(p2)

#         attention = tf.keras.layers.Dense(1, activation='tanh')(bilstm)
#         attention = tf.keras.layers.Flatten()(attention)
#         attention = tf.keras.layers.Activation('softmax')(attention)
#         attention = tf.keras.layers.RepeatVector(64 * 2)(attention)
#         attention = tf.keras.layers.Permute((2, 1))(attention)

#         x = tf.keras.layers.multiply([bilstm, attention])
#         x = tf.keras.layers.Lambda(lambda xx: tf.keras.backend.sum(xx, axis=1))(x)

#         output = tf.keras.layers.Dense(3, activation='softmax')(x)
#         model = tf.keras.models.Model(inputs=inputs, outputs=output)


#         inputs = tf.keras.Input(shape=(768 *3,))
#         h1 = tf.keras.layers.Dense(128, activation='relu')(inputs)
#         h2 = tf.keras.layers.Dense(128, activation='relu')(h1)

#         h2 = tf.keras.layers.Reshape((2, 64))(h2)
#         bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(h2)
        
#         attention = tf.keras.layers.Dense(1, activation='tanh')(bilstm)
#         attention = tf.keras.layers.Flatten()(attention)
#         attention = tf.keras.layers.Activation('softmax')(attention)
#         attention = tf.keras.layers.RepeatVector(64)(attention)
#         attention = tf.keras.layers.Permute((2, 1))(attention)

#         x = tf.keras.layers.multiply([h2, attention])
#         x = tf.keras.layers.Lambda(lambda xx: tf.keras.backend.sum(xx, axis=1))(x)

#         x = tf.keras.layers.Dense(32, activation='relu')(x)
#         output = tf.keras.layers.Dense(3, activation='softmax')(x)
#         model = tf.keras.models.Model(inputs=inputs, outputs=output)


        title_inputs = tf.keras.Input(shape=(768,))
        content_inputs = tf.keras.Input(shape=(768,))
        attention = tf.keras.layers.Attention()([title_inputs, content_inputs])
        x = tf.keras.layers.Flatten()(attention)
        x = tf.keras.layers.Dense(32, activation='relu')(x)
        output = tf.keras.layers.Dense(3, activation='softmax')(x)
        model = tf.keras.models.Model(inputs=[title_inputs, content_inputs], outputs=output)

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        model.fit([X_train[:, :768], X_train[:, 768: 768 * 2]], y_train, validation_data=([X_valid[:, :768], X_valid[:, 768: 768 * 2]], y_valid), callbacks=[early_stop], batch_size=32, epochs=20, verbose=0)

        y_pred = model.predict([X_valid[:, :768], X_valid[:, 768: 768 * 2]])
        y_valid = np.argmax(y_valid, axis=1)
        y_pred = np.argmax(y_pred, axis=1)
        valid_score = f1_score(y_valid, y_pred, average='macro')
        temp_model.append(model)
        temp_score.append(valid_score)
    best_idx = np.argmax(temp_score)
    model_list.append(temp_model[best_idx])
    print(temp_score[best_idx])
    score_list.append(temp_score[best_idx])
print(np.mean(score_list))

0
0.5975266786782165
1
0.5575967144511238
2
0.5356636282437143
3


/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.5198632326518015
4
0.5669043357333513
0.5555109179516415


In [10]:
y_pred = None
for model in model_list:
    if y_pred is None:
        y_pred = model.predict(X_test)
    else:
        y_pred = y_pred + model.predict(X_test)
y_pred = np.argmax(y_pred / 5, axis=1)

In [11]:
submission_df = pd.DataFrame()
submission_df['id'] = test_df['id'].values
submission_df['label'] = y_pred
submission_df.to_csv('./submission.csv', index=False)

In [8]:
fine_tune_submission_df = pd.read_csv('/Users/yitinglin/Desktop/submission.csv')
fine_tune_label = fine_tune_submission_df['label'].values + 1
feature_extract_label = submission_df['label'].values
label = []
for i in range(len(feature_extract_label)):
    if feature_extract_label[i] == 0:
        label.append(feature_extract_label[i])
        continue
    if feature_extract_label[i] != fine_tune_label[i]:
        label.append(fine_tune_label[i])
    else:
        label.append(feature_extract_label[i])
submission_df['label'] = label

In [9]:
submission_df.to_csv('./submission.csv', index=False)